In [1]:
import praw
import pandas as pd
import datetime as dt
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

reddit = praw.Reddit(client_id='',
                     client_secret='',
                     user_agent='')


In [2]:
print(reddit.read_only)  # Output: True

True


In [92]:
subreddit = reddit.subreddit('Conservative')

In [106]:
for submission in subreddit.hot(limit=10):
    print(submission.title)


President Benjamin Harrison explains why the Left hates the traditional family
Al Sharpton taking more than a million dollars from his “charity’ as salary.
Only a Muslim Democrat can get away with this s***: Ilhan Omar funneled another $150K to alleged lover’s consulting group
Obama Granted Clemency to Terrorists and Traitors, But We’re Supposed to be Angry at Trump's Pardons
Department of Justice Threatens to Arrest Oregon Judges Who Try to Keep Immigration Agents From Enforcing the Law
Ilhan Omar funneled another $150K to alleged lover’s consulting group
Poll: 53% of Black Voters, 60% of Other Minority Voters Say Media Are Trying to Help Impeach Trump
Democratic Representative Says He Will Vote NO on Impeachment, Reveals Other Dem Lawmakers “Quietly, Privately” Concerned About Schiff Show Trials
Portland - Deranged Anti-ICE Protester Sentenced To 3 Years In Prison After Attacking Disabled Man. Last Week another antifa terrorist received a six year prison sentence after wacking a guy 

In [94]:
top_subreddit = subreddit.top(limit=5000)


In [95]:
topics_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[]}


In [96]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)



In [97]:
topics_data = pd.DataFrame({ "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[]})


In [98]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)


In [99]:
_timestamp = topics_data["created"].apply(get_date)


In [100]:
topics_data = topics_data.assign(timestamp = _timestamp)


In [101]:
topics_data.head()

,title,score,id,url,comms_num,created,body,timestamp
0,So let me get this straight...,19572,5iugd6,http://i.imgur.com/L0p4pQA.jpg,3200,1.482010e+09,,2016-12-17 16:27:38
1,Why we won,15250,5c3xah,https://pbs.twimg.com/media/Cw1oX9aUQAA-rBU.jpg,1645,1.478756e+09,,2016-11-10 00:41:04
2,Ben Shapiro on the 4th of July...,14017,6la6tn,https://i.redd.it/ri98l0fdjn7z.jpg,1257,1.499236e+09,,2017-07-05 02:28:00
3,"Well, she's a guy, so...",12066,60k0bb,https://i.redd.it/kvriz780bnmy.jpg,1431,1.490080e+09,,2017-03-21 03:13:52
4,Reddit Admin u/spez Admits of Editing Users Co...,11301,5el3n6,https://i.redd.it/jt4fsjpargzx.png,610,1.479980e+09,,2016-11-24 04:29:49


In [102]:
len(topics_data)

998

In [103]:
topics_data.describe()

,score,comms_num,created
count,998.000000,998.000000,9.980000e+02
mean,2928.532064,299.720441,1.547567e+09
std,1384.685923,253.788856,2.531359e+07
min,1859.000000,26.000000,1.434479e+09
25%,2167.000000,153.250000,1.541131e+09
50%,2545.000000,234.000000,1.554972e+09
75%,3185.250000,352.000000,1.564978e+09
max,19572.000000,3200.000000,1.574026e+09


In [104]:
max(topics_data.timestamp)

Timestamp('2019-11-17 16:22:21')

In [105]:
min(topics_data.timestamp)

Timestamp('2015-06-16 14:31:13')

In [29]:

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)


In [33]:
def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format( \
                subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

In [87]:
def give_me_intervals(start_at, end_at, number_of_days_per_interval = 3):
            
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)
    end = start_at + period
    yield (int(start_at), int(end))
    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)
## test out the solution,
start_at = math.floor(\
     (datetime.utcnow() - timedelta(days=365)).timestamp())
list(give_me_intervals(start_at, 7))


[(1542528806, 1542788006)]

In [89]:
subreddit = 'Conservative'
from datetime import date
end_at=date.fromisoformat('2018-11-06')
start_at=(end_at) - timedelta(days=30)
end_at_unix = time.mktime(end_at.timetuple())
start_at_unix = time.mktime(start_at.timetuple())

posts = []
for interval in give_me_intervals(start_at_unix, end_at_unix, 7):
    pulled_posts = pull_posts_for(
        subreddit, interval[0], interval[1])
    
    posts.extend(pulled_posts)
    time.sleep(.500)
## ~ 4306
print(len(posts))
## ~ 4306
print(len(np.unique([ post['id'] for post in posts ])))

9219
9202


In [108]:
election_dates = ['2018-11-06','2016-11-08',
                 '2014-11-04','2012-11-06']

In [110]:
subreddits = ['Liberal','Conservative','moderatepolitics']

In [142]:
posts = []
z=0
for x in election_dates:

    for y in subreddits:
        print(x, " : ", y)
        subreddit = y
        end_at=date.fromisoformat(x)
        start_at=(end_at) - timedelta(days=30)
        end_at_unix = time.mktime(end_at.timetuple())
        start_at_unix = time.mktime(start_at.timetuple())

        temp_posts = []
        for interval in give_me_intervals(start_at_unix, end_at_unix, 7):
            pulled_posts = pull_posts_for(
                subreddit, interval[0], interval[1])

            temp_posts.extend(pulled_posts)
            time.sleep(.500)
        
        if z==0:
            df = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            z=1
        else:
            df2 = pd.DataFrame({'year':x,
                              'subreddit':y,
                              'posts':temp_posts})
            
            df=df.append(df2,ignore_index=True)

            
   


2018-11-06  :  Liberal
2018-11-06  :  Conservative
2018-11-06  :  moderatepolitics
2016-11-08  :  Liberal
2016-11-08  :  Conservative
2016-11-08  :  moderatepolitics
2014-11-04  :  Liberal
2014-11-04  :  Conservative
2014-11-04  :  moderatepolitics
2012-11-06  :  Liberal
2012-11-06  :  Conservative
2012-11-06  :  moderatepolitics


In [143]:
df.head()

,year,subreddit,posts
0,2018-11-06,Liberal,"{'id': '9m2dtt', 'created_utc': 1538884997, 'p..."
1,2018-11-06,Liberal,"{'id': '9m2f5d', 'created_utc': 1538885384, 'p..."
2,2018-11-06,Liberal,"{'id': '9m2ixb', 'created_utc': 1538886524, 'p..."
3,2018-11-06,Liberal,"{'id': '9m2kyh', 'created_utc': 1538887168, 'p..."
4,2018-11-06,Liberal,"{'id': '9m3ygv', 'created_utc': 1538905752, 'p..."


In [144]:
print(len(df))

27830


In [145]:
df['subreddit'].value_counts()

Conservative        24175
Liberal              2916
moderatepolitics      739
Name: subreddit, dtype: int64

In [146]:
df['year'].value_counts()

2018-11-06    10308
2012-11-06     7319
2016-11-08     5603
2014-11-04     4600
Name: year, dtype: int64

In [ ]:
## WARNING: REDDIT WILL THROTTLE YOU IF YOU ARE ANNOYING! BE KIND!

z=0
TIMEOUT_AFTER_COMMENT_IN_SECS = 1
posts_from_reddit = []

final_df = { "subreddit":[],
            "year":[],
            "title":[], 
            "score":[], 
            "id":[], "url":[],  
            "comms_num": [], 
            "created": [], 
            "body":[]}


for x in range(len(df)):
    submission = reddit.submission(id=df['posts'][x]['id'])
    posts_from_reddit.append(submission)
#    submission.comments.replace_more(limit=None)
#    for comment in submission.comments.list():
#        comments_from_reddit.append(comment)

    final_df["title"].append(submission.title)
    final_df["score"].append(submission.score)
    final_df["id"].append(submission.id)
    final_df["url"].append(submission.url)
    final_df["comms_num"].append(submission.num_comments)
    final_df["created"].append(submission.created)
    final_df["body"].append(submission.selftext)
    final_df['subreddit'].append(df['subreddit'][x])
    final_df['year'].append(df['year'][x])

    time.sleep(0.5)

## ~ 4306
print(len(final_df))
## ~ 35216
print(len(final_df))
